In [1]:
# https://github.com/justwjr/DSCI6007-student/blob/master/The%20Internet/lab.md
import os
import yaml
credentials = yaml.load(open(os.path.expanduser('~/api_cred.yml')))

In [2]:
credentials

{'twitter': {'consumer_key': 'ZFPIkvZvMxUchIQzV31qYOL7m',
  'consumer_secret': '8usCoOkboWMNS3EWpeUnj8GyFNmpi2R240kPy0XNKxR5gg8JOz',
  'token': '965746291-2Oz066vZS3B4V4dT5ixJDL4pCsfQ3hSZ1BtnFwPR',
  'token_secret': 'hiDFNNMp6LjodiZ4LxWBnHPwxuF1Bw9SeaPMMctYdVoPH'}}

In [3]:
credentials['twitter'].get('consumer_key')

'ZFPIkvZvMxUchIQzV31qYOL7m'

In [4]:
# https://github.com/ideoforms/python-twitter-examples/blob/master/twitter-trends.py

In [5]:
#!/usr/bin/python

#-----------------------------------------------------------------------
# twitter-trends
#  - lists the current global trending topics
#-----------------------------------------------------------------------

from twitter import *

#-----------------------------------------------------------------------
# load our API credentials 
#-----------------------------------------------------------------------
# config = {}
# execfile("config.py", config)

#-----------------------------------------------------------------------
# create twitter API object
#-----------------------------------------------------------------------
twitter = Twitter(auth = OAuth(credentials['twitter'].get('token'),
                               credentials['twitter'].get('token_secret'),
                               credentials['twitter'].get('consumer_key'), 
                               credentials['twitter'].get('consumer_secret'))
)

twitter

In [6]:
#-----------------------------------------------------------------------
# retrieve global trends.
# other localised trends can be specified by looking up WOE IDs:
#   http://developer.yahoo.com/geo/geoplanet/
# twitter API docs: https://dev.twitter.com/docs/api/1/get/trends/%3Awoeid
#-----------------------------------------------------------------------
results = twitter.trends.place(_id = 23424977)

print "US Trends"

for location in results:
    for trend in location["trends"]:
        print " - %s" % trend["name"]

US Trends
 - Sean Spicer
 - #IMarchWithLinda
 - #mondaymotivation
 - The Last Jedi
 - #NationalPieDay
 - #SchoolChoice
 - #MusicMonday
 - Pro Bowl
 - Gail Miller
 - Mike Groh
 - D-League
 - Tucson International Airport
 - Marco Rubio
 - Google Voice
 - Marie Harf
 - Player of the Week
 - Tidal
 - Flynn
 - Resident Evil 7
 - Father John Misty
 - Patti Cake
 - Tuscon Airport
 - Aetna
 - CALL ME BY YOUR NAME
 - DAPL
 - Yahoo Q4
 - Trump Organization
 - Wil Myers
 - #LawyersIn5Words
 - #MyPerfectDatingApp
 - #PressBriefing
 - #CMAD
 - #iHeartAustinMahone
 - #MakeSomeoneOlder
 - #SOTN17
 - #beliebersstayforever
 - #contentchat
 - #H2HChat
 - #IHandleChangeBy
 - #RoyalCaninDogShow
 - #ATLCollegeFair
 - #Emmys
 - #NewEdition


In [7]:
trends = [trend["name"] for trend in location["trends"] for location in results]
trends[:10]

[u'Sean Spicer',
 u'#IMarchWithLinda',
 u'#mondaymotivation',
 u'The Last Jedi',
 u'#NationalPieDay',
 u'#SchoolChoice',
 u'#MusicMonday',
 u'Pro Bowl',
 u'Gail Miller',
 u'Mike Groh']

In [8]:
import pandas as pd
df = pd.DataFrame(trends[:10])
df.head()

,0
0,Sean Spicer
1,#IMarchWithLinda
2,#mondaymotivation
3,The Last Jedi
4,#NationalPieDay


In [14]:
top10html = df.to_html()
top10html

u'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>0</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Sean Spicer</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>#IMarchWithLinda</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>#mondaymotivation</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>The Last Jedi</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>#NationalPieDay</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>#SchoolChoice</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>#MusicMonday</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td>Pro Bowl</td>\n    </tr>\n    <tr>\n      <th>8</th>\n      <td>Gail Miller</td>\n    </tr>\n    <tr>\n      <th>9</th>\n      <td>Mike Groh</td>\n    </tr>\n  </tbody>\n</table>'

In [16]:
myfile = open('top10.html', 'w')
myfile.write(top10html)
# for location in results:
#     for i, trend in enumerate(location["trends"]):
#         if i < 10:
#             myfile.write("%s\n" %trend["name"])
myfile.close()

In [10]:
myfile = open('top10.txt', 'w')
for location in results:
    for i, trend in enumerate(location["trends"]):
        if i < 10:
            myfile.write("%s\n" %trend["name"])
myfile.close()

In [11]:
ls

HTML/                            error.html
HTTP/                            html-intro/
Internet-Lab-Justin-Srini.ipynb  index.html
Internet-Lab-Justin-Srini.py     lab.md
README.md                        top10.txt


In [12]:
import time
import boto
from boto.s3.connection import Location

#
# create a couple of strings with our very minimal web content
#
index_html = "<html><head><title>My Top Twitter Trends</title></head><body>"+ top10html +"</body></html>"

error_html = """
<html>
  <head><title>Something is wrong</title></head>
  <body><h2>Something is terribly wrong with my S3-based website</h2></body>
</html>"""

# create a connection to S3
conn = boto.connect_s3(host='s3-us-west-1.amazonaws.com')
# create a bucket and make it publicly readable
# website_bucket = conn.create_bucket('justw150',
#                                     location=Location.USWest,
#                                     policy='public-read')
website_bucket = conn.get_bucket('justw150')

# upload our HTML pages and make sure they are publicly readable
# also make sure Content-Type is set to text/html
index_key = website_bucket.new_key('index.html')
index_key.content_type = 'text/html'
index_key.set_contents_from_string(index_html, policy='public-read')
error_key = website_bucket.new_key('error.html')
error_key.content_type = 'text/html'
error_key.set_contents_from_string(error_html, policy='public-read')

# now set the website configuration for our bucket
website_bucket.configure_website('index.html', 'error.html')

time.sleep(5)

# now get the website configuration, just to check it
print website_bucket.get_website_configuration()

{u'WebsiteConfiguration': {u'IndexDocument': {u'Suffix': u'index.html'}, u'ErrorDocument': {u'Key': u'error.html'}}}


In [13]:
# http://justw150.s3-website-us-west-1.amazonaws.com/